https://stackoverflow.com/questions/35919020/whats-the-difference-of-name-scope-and-a-variable-scope-in-tensorflow

In [1]:
import tensorflow as tf

# Tensorflow Tensor naming convention

The name of a Tensor is the concatenation of

1. the name of the operation that produced it,
2. a colon (:), and
3. the index of that tensor in the outputs of the operation that produced it.

Therefore the tensor named `foo:2` is the output of the op named `foo` at position `2` (with indices starting from zero).

In [2]:
a = tf.constant([1, 2, 3, 4, 5, 6])
s1, s2, s3 = tf.split(a, 3)
print(s1.name)
print(s2.name)
print(s3.name)

split:0
split:1
split:2


# tf.Variable 

Using the `tf.Variable` constructor which will create a new variable every time it is called (and potentially add a suffix to the variable name if a variable with such name already exists).

In [3]:
a = tf.Variable(1, name='my_var', dtype=tf.int64) # create a variable using tf.Variable constructor
print(a.name)
a = tf.Variable(2, name='my_var', dtype=tf.int64) # creating another variable using the same name 
                                                       # will add a suffix to the variable name 
                                                       # to distinguish between the two variables
print(a.name)

my_var:0
my_var_1:0


# tf.get_variable()

The method `tf.get_variable()` can be used to create a new variable or retrieve the one that was created before.

In [4]:
a = tf.get_variable(name='my_variable', shape=(4,), dtype=tf.float32) # create a variable using tf.get_variable()
print(a.name)

W0719 17:20:34.056622 11900 deprecation.py:506] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


my_variable:0


In [5]:
# b = tf.get_variable(name='my_variable') # Error, variable 'my_variable' already exists. 
                                          # To reuse a variable that was created before, 
                                          # we have to use variable_scope.

`tf.get_variable()` prefixes the name with the current variable scope and performs reuse checks.

In [6]:
with tf.variable_scope('my_scope'):
    a = tf.get_variable(name='my_variable', shape=(1,), dtype=tf.float32)
    print(a.name)

my_scope/my_variable:0


In [7]:
with tf.variable_scope('my_scope', reuse=True): # reuse=True or reuse=tf.AUTO_REUSE
    b = tf.get_variable(name='my_variable')
    print(b.name)

my_scope/my_variable:0


In [8]:
assert a==b

# Name scope vs Variable scope

Both scopes have the same effect on all operations as well as variables created using `tf.Variable`, i.e., the scope will be added as a prefix to the operation or variable name.

However, `tf.name_scope` is ignored by `tf.get_variable()`.

In [9]:
# using tf.name_scope
with tf.name_scope("name_scope"):
    v1 = tf.get_variable("get_variable", [1], dtype=tf.float32)
    v2 = tf.Variable(1, name="Variable", dtype=tf.float32)
    a = tf.add(v1, v2)
print(v1.name) # name_scope is ignored
print(v2.name)
print(a.name)

get_variable:0
name_scope/Variable:0
name_scope/Add:0


In [10]:
# using tf.variable_scope
with tf.variable_scope("variable_scope"):
    v1 = tf.get_variable("get_variable", [1], dtype=tf.float32)
    v2 = tf.Variable(1, name="Variable", dtype=tf.float32)
    a = tf.add(v1, v2)
print(v1.name)
print(v2.name)
print(a.name)

variable_scope/get_variable:0
variable_scope/Variable:0
variable_scope/Add:0


<b>Use case</b>: allows us to easily share variables across different parts of the program, even within different name scopes.

In [11]:
with tf.name_scope("foo"):
    with tf.variable_scope("var_scope"):
        v = tf.get_variable("var", [1])
with tf.name_scope("bar"):
    with tf.variable_scope("var_scope", reuse=True):
        v1 = tf.get_variable("var", [1])
assert v1 == v
print(v.name)
print(v1.name)

var_scope/var:0
var_scope/var:0
